# Configuração do Ambiente para YOLO v11

In [ ]:
# Instalar Ultralytics (YOLO v11)
!pip install ultralytics

In [ ]:
# Verificar a versão do YOLO instalada
import ultralytics
ultralytics.checks()

In [ ]:
# Verificar a disponibilidade da GPU
!nvidia-smi

# Preparação dos Dados para YOLO

In [ ]:
# prompt: add a code to extract /content/individual_urban_tree_crown_detection.zip to a folder of the same name

import zipfile
import os

zip_file_path = '/content/individual_urban_tree_crown_detection.zip'
extract_folder_name = 'individual_urban_tree_crown_detection'
extract_path = os.path.join('/content', extract_folder_name)

# Create the directory if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted {zip_file_path} to {extract_path}")

In [ ]:
import os
import shutil
from PIL import Image

# --- CONFIGURE AQUI ---
# Caminho base do dataset descompactado (assumindo que já foi descompactado para /home/ubuntu/dataset/individual_urban_tree_crown_detection)
base_dataset_path = "/content/individual_urban_tree_crown_detection/individual_urban_tree_crown_detection"

# 1. Caminho para a pasta com os patches de imagem RGB
source_images_dir = os.path.join(base_dataset_path, "rgb")
# 2. Caminho para a pasta com as anotações (.txt) em formato de PIXELS
source_labels_dir = os.path.join(base_dataset_path, "bbox_txt")
# 3. Caminho para a pasta com as listas de arquivos (contendo as pastas 0, 1, 2, 3, 4)
split_files_dir = os.path.join(base_dataset_path, "img_list")
# 4. Escolha qual 'fold' (pasta de 0 a 4) usar para a divisão
fold_to_use = 0  # Mude para 1, 2, 3, ou 4 para usar outros folds
# 5. Extensão dos seus arquivos de imagem (ex: '.jpg', '.png')
image_extension = '.png'
# ---------------------

# Nome da pasta base para o dataset formatado para o YOLO
base_dir = "/content/yolo_dataset"

# Limpa e cria a estrutura de pastas do zero
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
os.makedirs(os.path.join(base_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'labels/val'), exist_ok=True)
print(f"Estrutura de pastas criada em: {base_dir}")

def get_filenames_from_list(filepath):
    if not os.path.exists(filepath):
        print(f"ERRO: Arquivo de lista não encontrado em: {filepath}")
        return []
    with open(filepath, 'r') as f:
        return [line.strip() for line in f.readlines() if line.strip()]

train_list_path = os.path.join(split_files_dir, str(fold_to_use), 'train.txt')
val_list_path = os.path.join(split_files_dir, str(fold_to_use), 'val.txt')
train_basenames = get_filenames_from_list(train_list_path)
val_basenames = get_filenames_from_list(val_list_path)

print(f"\nUsando Fold: {fold_to_use}")
print(f"Imagens de treino a serem processadas: {len(train_basenames)}")
print(f"Imagens de validação a serem processadas: {len(val_basenames)}")

def process_and_copy_files(basename_list, source_img_dir, source_lbl_dir, dest_img_dir, dest_lbl_dir, img_ext):
    """
    Copia as imagens e processa os arquivos de anotação para o formato YOLO.
    Assume que o formato de origem é: x_min y_min x_max y_max (por linha)
    Converte para: class_id center_x_norm center_y_norm width_norm height_norm
    """
    processed_count = 0
    for base_filename in basename_list:
        img_filename = base_filename
        # O nome do arquivo de label é o mesmo do arquivo de imagem, mas com extensão .txt
        label_filename = f"{os.path.splitext(base_filename)[0]}.txt"

        source_img_path = os.path.join(source_img_dir, img_filename)
        source_label_path = os.path.join(source_lbl_dir, label_filename)
        dest_img_path = os.path.join(dest_img_dir, img_filename)
        dest_label_path = os.path.join(dest_lbl_dir, label_filename)

        if os.path.exists(source_img_path) and os.path.exists(source_label_path):
            # Obter as dimensões da imagem
            try:
                with Image.open(source_img_path) as img:
                    IMG_WIDTH, IMG_HEIGHT = img.size
            except Exception as e:
                print(f"ERRO ao ler imagem {source_img_path}: {e}")
                continue

            # 1. Copia o arquivo de imagem
            shutil.copy(source_img_path, dest_img_path)

            # 2. Converte o arquivo de anotação
            normalized_labels = []
            with open(source_label_path, 'r') as f_in:
                for line in f_in:
                    parts = line.strip().split()
                    if len(parts) < 4: continue # Ignora linhas mal formatadas

                    # Assume o formato x_min, y_min, x_max, y_max
                    x_min, y_min, x_max, y_max = map(float, parts[:4])

                    # Calcula o centro, largura e altura em pixels
                    box_width = x_max - x_min
                    box_height = y_max - y_min
                    center_x = x_min + (box_width / 2)
                    center_y = y_min + (box_height / 2)

                    # Normaliza as coordenadas
                    norm_center_x = center_x / IMG_WIDTH
                    norm_center_y = center_y / IMG_HEIGHT
                    norm_width = box_width / IMG_WIDTH
                    norm_height = box_height / IMG_HEIGHT

                    # Adiciona o ID da classe (0 para 'tree')
                    # Formato: class_id center_x center_y width height
                    normalized_labels.append(f"0 {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")

            # Salva o novo arquivo de anotação normalizado
            with open(dest_label_path, 'w') as f_out:
                f_out.write("\n".join(normalized_labels))

            processed_count += 1
        else:
            if not os.path.exists(source_img_path):
                print(f"AVISO: Imagem '{img_filename}' não encontrada em {source_img_dir}.")
            if not os.path.exists(source_label_path):
                print(f"AVISO: Anotação '{label_filename}' não encontrada em {source_labels_dir}.")
            print(f"--> O par para '{base_filename}' será ignorado.")

    return processed_count

# Processa e copia os arquivos de treino
print("\nProcessando arquivos de TREINO...")
train_copied = process_and_copy_files(train_basenames, source_images_dir, source_labels_dir, os.path.join(base_dir, 'images/train'), os.path.join(base_dir, 'labels/train'), image_extension)
print(f"{train_copied} pares de imagem/anotação processados para o conjunto de treino.")

# Processa e copia os arquivos de validação
print("\nProcessando arquivos de VALIDAÇÃO...")
val_copied = process_and_copy_files(val_basenames, source_images_dir, source_labels_dir, os.path.join(base_dir, 'images/val'), os.path.join(base_dir, 'labels/val'), image_extension)
print(f"{val_copied} pares de imagem/anotação processados para o conjunto de validação.")

# Verificação final
print("\n--- Verificação Final ---")
train_img_count = len(os.listdir(os.path.join(base_dir, 'images/train')))
train_lbl_count = len(os.listdir(os.path.join(base_dir, 'labels/train')))
val_img_count = len(os.listdir(os.path.join(base_dir, 'images/val')))
val_lbl_count = len(os.listdir(os.path.join(base_dir, 'labels/val')))
print(f"Imagens de treino: {train_img_count} | Anotações de treino: {train_lbl_count}")
print(f"Imagens de validação: {val_img_count} | Anotações de validação: {val_lbl_count}")

if train_img_count == train_lbl_count and val_img_count == val_lbl_count and train_img_count > 0 and val_img_count > 0:
    print("\nOrganização e normalização do dataset concluídas com sucesso!")
else:
    print("\nERRO: A contagem de imagens e anotações não corresponde. Verifique os avisos.")

# Configuração do Arquivo YAML do Dataset

In [ ]:
# Criar o arquivo data.yaml para o YOLO
yaml_content = f"""\
path: {base_dir}  # Caminho para a pasta raiz do dataset
train: images/train  # Caminho relativo para as imagens de treino
val: images/val    # Caminho relativo para as imagens de validação

# Número de classes
nc: 1

# Nomes das classes
names: ['tree']
"""
with open(os.path.join(base_dir, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("Arquivo data.yaml criado com sucesso:")
print(yaml_content)

# Treinamento do Modelo YOLOv11

In [ ]:
from ultralytics import YOLO

# Carregar um modelo pré-treinado (ex: yolov8n.pt para YOLOv8 nano, ou yolov11.pt se disponível)
# Para YOLOv11, você pode precisar baixar o modelo correspondente ou usar um modelo YOLOv8 como base
# Se yolov11.pt não estiver disponível, tente yolov8n.pt ou outro modelo YOLOv8
model = YOLO('yolo11s.pt')  # Substitua por 'yolov11.pt' se tiver o modelo

# Treinar o modelo
results = model.train(data=os.path.join(base_dir, "data.yaml"), epochs=40, imgsz=512, batch=16)

print("Treinamento concluído! Resultados salvos em runs/detect/train")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# prompt: gere um codigo para copiar a pasta /content/runs para o MeuDrive, porém se a pasta já existir, ele cria outra igual com numeração, ex: testes1, testes2, etc.

import os
import shutil

source_dir = '/content/runs'
destination_base_dir = '/content/drive/MyDrive/yolov11' # Base path in Google Drive

# Ensure the base destination directory exists
if not os.path.exists(destination_base_dir):
    os.makedirs(destination_base_dir)

# Check if the source directory exists
if not os.path.exists(source_dir):
    print(f"Source directory does not exist: {source_dir}")
else:
    # Determine the destination path, checking for existing folders
    dest_dir = os.path.join(destination_base_dir, 'runs')
    counter = 1
    while os.path.exists(dest_dir):
        dest_dir = os.path.join(destination_base_dir, f'runs_{counter}')
        counter += 1

    # Copy the directory
    try:
        shutil.copytree(source_dir, dest_dir)
        print(f"Successfully copied '{source_dir}' to '{dest_dir}'")
    except Exception as e:
        print(f"Error copying directory: {e}")

# Validação do Modelo

In [ ]:
# Validar o modelo treinado
metrics = model.val()  # Valida o modelo nos dados de validação definidos em data.yaml

print("Métricas de validação:")
print(metrics)

# Inferência (Detecção em Novas Imagens)

In [ ]:
# Realizar inferência em uma imagem de teste (exemplo)
# Substitua 'path/to/your/test_image.png' pelo caminho de uma imagem que você queira testar
# Certifique-se de que a imagem esteja acessível no ambiente
test_image_path = os.path.join(base_dir, 'images/val/132.png') # Exemplo: usando uma imagem do conjunto de validação

if os.path.exists(test_image_path):
    results = model(test_image_path)  # Executa a inferência

    # Mostrar os resultados (opcional)
    for r in results:
        im_bgr = r.plot()  # plotagem BGR da imagem com caixas e rótulos
        # Para exibir no Jupyter/Colab, você pode converter para RGB e usar matplotlib
        import cv2
        import matplotlib.pyplot as plt
        im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
        plt.imshow(im_rgb)
        plt.axis('off')
        plt.show()
else:
    print(f"Imagem de teste não encontrada: {test_image_path}")